# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [2]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [3]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [ ]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [5]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [6]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [7]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [8]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [9]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/
https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/
https://edwarddonner.com/2024/11/13/llm-engineering-resources/
https://edwarddonner.com/2024/11/13/ll

In [10]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [11]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/blog/smolagents',
 '/microsoft/phi-4',
 '/hexgrad/Kokoro-82M',
 '/deepseek-ai/DeepSeek-V3',
 '/NovaSky-AI/Sky-T1-32B-Preview',
 '/openbmb/MiniCPM-o-2_6',
 '/models',
 '/spaces/hexgrad/Kokoro-TTS',
 '/spaces/JeffreyXiang/TRELLIS',
 '/spaces/lllyasviel/iclight-v2',
 '/spaces/stabilityai/stable-point-aware-3d',
 '/spaces/FaceOnLive/Face-Search-Online',
 '/spaces',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/NovaSky-AI/Sky-T1_data_17k',
 '/datasets/DAMO-NLP-SG/multimodal_textbook',
 '/datasets/cfahlgren1/react-code-instructions',
 '/datasets/HumanLLMs/Human-Like-DPO-Dataset',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer

In [12]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [13]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [14]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
NEW
smolagents - a smol library to build gr

In [15]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [16]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [17]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'jobs page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'github page', 'url': 'https://github.com/huggingface'}, {'type': 'linkedin page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'twitter page', 'url': 'https://twitter.com/huggingface'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nNEW\nsmolagents - a smol library to build great agents\nUse models from the HF Hub in LM Studio\nUse Ollama with GGUF Models from the HF Hub\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nTrending on\nthis week\nModels\nmicrosoft/phi-4\nUpdated\n7 days ago\n•\n72.3k\n•\n1.29k\nhexgrad/Kokoro-82M\nUpdated\n1 day ago\n•\n15.6k\n•\n1.39k\ndeepseek-ai/DeepSeek-V3\nUpdated\n16 days ago\n•\n132k\n•\n1.87k\nNovaSky-AI/Sky-T1-32B-Preview\nUpdated\n2 days ago\n•\n2.88k\n•\n361\nopenbmb/MiniCPM-o-2_6\nUpdated\n35 minut

In [18]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [19]:
create_brochure("HuggingFace", "https://huggingface.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.com/'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.com/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.com/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.com/blog'}, {'type': 'contact page', 'url': 'https://huggingface.com/join'}]}


```markdown
# Hugging Face: The AI Community Building the Future

## Overview
Hugging Face is a collaboration platform that empowers the machine learning community to create, discover, and share models, datasets, and applications. With over *400,000 models* and *100,000 datasets*, Hugging Face has become the go-to hub for machine learning professionals, researchers, and enthusiasts worldwide.

## Our Mission
At Hugging Face, we believe in building the future of AI through community collaboration. Our mission is to accelerate machine learning adoption by providing open-source tools and platforms that make it easier to create and deploy AI solutions.

## Company Culture
Hugging Face thrives on a vibrant and inclusive company culture that emphasizes collaboration and innovation. We are committed to supporting a diverse range of voices and ideas within the AI community while fostering an environment that encourages creativity and multidisciplinary teamwork.

- **Open Source First**: Our development philosophy prioritizes open-source contributions. We work closely with our community to expand our toolset and improve our offerings.
- **Community-Driven**: We strive to engage with our users through forums, social media, and direct collaborations to ensure our platform meets their needs.

## Customers
More than *50,000 organizations* leverage Hugging Face capabilities. Our customers include industry leaders, research institutions, and innovative startups such as:

- **Amazon Web Services**
- **Meta**
- **Google**
- **Microsoft**
- **Intel**

These organizations utilize our vast model repository, user-friendly API, and enterprise-grade solutions to build and deploy their AI applications.

## Career Opportunities
Join the Hugging Face family! We are constantly on the lookout for passionate individuals eager to contribute to the evolution of AI. Whether you are a data scientist, software engineer, researcher, or product manager, we offer diverse career paths and growth opportunities in an inclusive environment.

### Benefits of Working with Us:
- **Flexible Work Environment**: We understand the importance of work-life balance.
- **Impactful Work**: Contribute to groundbreaking AI projects that have a real-world impact.
- **Growth Opportunities**: Engage in continuous learning and professional development.

Visit our [Careers Page](#) for current openings and to learn more about what makes Hugging Face an excellent place to work!

## Join Us
Whether you're looking to collaborate with cutting-edge technology, seeking career opportunities, or interested in supporting the next generation of machine learning, Hugging Face invites you to be part of our journey to transform the AI landscape.

For more information, please visit [Hugging Face](https://huggingface.co).
```


## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [20]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [21]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Brochure

**Hugging Face: The AI Community Building the Future**

---

## About Us

Hugging Face is a collaborative platform at the forefront of AI and machine learning. We empower developers, researchers, and organizations to create, discover, and contribute to innovative models, datasets, and applications. With over 400k models and 100k datasets available, we are committed to advancing the future of machine learning through open-source collaboration.

---

## Our Offerings

- **Models:** Access a diverse range of state-of-the-art models including `microsoft/phi-4`, `hexgrad/Kokoro-82M`, and `deepseek-ai/DeepSeek-V3`.

- **Datasets:** Explore a rich repository of datasets for various domains, facilitating cutting-edge research and application development.

- **Spaces:** Create and share applications that leverage machine learning models in unique and collaborative ways.

- **Enterprise Solutions:** Tailored for teams looking for robust ML infrastructure with advanced security, dedicated support, and enterprise-grade controls.

---

## Customer Base

We proudly serve over **50,000 organizations**, including industry leaders like:

- **Google** (913 models)
- **Amazon Web Services** (20 models)
- **Microsoft** (353 models)
- **Meta** (2,060 models)

Our customers include both enterprises and academic institutions, aiming to harness the power of AI technologies.

---

## Company Culture

At Hugging Face, we pride ourselves on being a community-driven organization. Our culture encourages:

- **Collaboration:** We believe in the power of community and the free exchange of ideas.
- **Innovation:** Team members are encouraged to explore new technologies and methodologies.
- **Inclusivity:** We promote a diverse and inclusive environment that values different perspectives.

---

## Careers at Hugging Face

We are always on the lookout for talented and passionate individuals who are excited about AI and machine learning. Joining our team means working on groundbreaking technologies and having the opportunity to make a significant impact on the future of AI.

**Current Openings:** Visit our [Jobs Page](https://huggingface.co/jobs) to learn more about career opportunities at Hugging Face.

---

## Join Us!

Whether you're a developer, researcher, or just passionate about AI, we invite you to join our community. Explore our [platform](https://huggingface.co) to get started!

---

**Contact Us:**  
- **Website:** [huggingface.co](https://huggingface.co)  
- **Social Media:** [Twitter](https://twitter.com/huggingface) | [LinkedIn](https://linkedin.com/company/huggingface)

**Together, let's build the future of AI!**

In [22]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}]}


# Hugging Face: The Future of AI

## Overview
Hugging Face is a leading platform for the machine learning community, focused on building innovations for the future of artificial intelligence. With state-of-the-art models, datasets, and tools, Hugging Face brings together developers, researchers, and companies to create, discover, and collaborate on ML applications.

## What We Offer
- **Model Hub**: Access to over **400,000+ models** across various modalities including text, images, and audio.
- **Datasets**: A vast repository of **100,000+ datasets** optimized for a variety of ML tasks.
- **Spaces**: A collaborative space for building and sharing applications, innovations, and learning resources.
- **Enterprise Solutions**: Tailored services with enterprise-grade security and support for over **50,000+ organizations**, including big names like Google, Microsoft, and Amazon Web Services.

## Company Culture
At Hugging Face, we pride ourselves on being part of a **community-driven culture**. We believe in open-source collaboration, allowing individuals and organizations to come together to enhance AI research and applications. Our team is passionate about **innovation, inclusivity, and transparency**, ensuring that everyone—from independent researchers to big enterprises—can benefit from our tools and contributions.

## Success Stories
Hugging Face has partnered with some of the most innovative companies worldwide, creating solutions that push the boundaries of what's possible in machine learning. Our models and datasets are used by organizations across various sectors, contributing to advancements in AI technologies and applications.

## Careers at Hugging Face
Join us in shaping the future of AI! At Hugging Face, we are constantly on the lookout for talented individuals who are passionate about machine learning and want to be part of a dynamic team. We offer various roles across engineering, data science, and community engagement.
- **Inclusive Work Environment**: We foster a workplace where ideas are shared freely, and creativity is encouraged.
- **Continuous Learning**: We invest in the continuous growth of our team members through workshops, learning sessions, and collaborations.

## Connect with Us
To learn more about our offerings, get involved with the community, or explore career opportunities, visit our website at [Hugging Face](https://huggingface.co). 

---

**Hugging Face**: Building a strong community around AI and transforming the landscape of machine learning together!

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>

In [49]:
link_system_prompt = "You are provided with a list of links found on a Dutch webpage. \
You are able to decide which of the links include general information about houses (like floors or 'woningzoekers'), not specific appartments or houses.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "woningaanbod", "url": "https://crossroadsamsterdam.nl/woningaanbod/"},
        {"type": "woningaanbod - verdieping 0 ": "url": "https://crossroadsamsterdam.nl/woningaanbod/verdieping-0/"}
    ]
}
"""
link_system_prompt += "You should avoid these links of specific appartments or houses:"
link_system_prompt += """
{
    "links": [
        {"type": "woningaanbod - verdieping 0  - appartement 6310' ": "url": "https://crossroadsamsterdam.nl/woningaanbod/verdieping-27/#6310"}
    ]
}
"""

def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links that contain information about the appartments, respond with the full https URL in JSON format. \
Do not include general information of the building, only of specific appartments or houses.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [50]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

cross = Website("https://crossroadsamsterdam.nl/")
cross.links


['https://crossroadsamsterdam.nl',
 'https://crossroadsamsterdam.nl/',
 'https://crossroadsamsterdam.nl/locatie/',
 'https://crossroadsamsterdam.nl/woningaanbod/',
 'https://crossroadsamsterdam.nl/voordelen/',
 'https://crossroadsamsterdam.nl/veelgesteldevragen/',
 'https://crossroadsamsterdam.nl/contact/',
 'https://account.crossroadsamsterdam.nl/#aanmelden',
 'https://crossroadsamsterdam.nl/woningaanbod',
 'https://account.crossroadsamsterdam.nl/#aanmelden',
 'https://crossroadsamsterdam.nl/veelgesteldevragen/',
 'https://crossroadsamsterdam.nl/contact',
 'https://crossroadsamsterdam.nl/cookies',
 'https://www.door.business']

In [51]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

def get_more_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    print("Found links:", json.loads(result)["links"])
    # do another iteration of the found links to see if there are more links to be found
    for link in json.loads(result)["links"]:
        print(link)
        website = Website(link["url"])
        print("Getting links from", link["url"])
        links = get_links(link["url"])
        print("Found links:", links)
        for link in links["links"]:
            if link not in json.loads(result)["links"]:
                print("Adding link", link)
                result["links"].append(link)
    return json.loads(result)


In [52]:
get_links("https://crossroadsamsterdam.nl/")




{'links': []}

In [48]:
get_links("https://crossroadsamsterdam.nl/woningaanbod/")

website = Website("https://crossroadsamsterdam.nl/woningaanbod/")
website.links


{'links': [{'type': 'woningaanbod - verdieping 27 - appartement 6309',
   'url': 'https://crossroadsamsterdam.nl/woningaanbod/verdieping-27/#6309'},
  {'type': 'woningaanbod - verdieping 27 - appartement 6310',
   'url': 'https://crossroadsamsterdam.nl/woningaanbod/verdieping-27/#6310'},
  {'type': 'woningaanbod - verdieping 27 - appartement 6311',
   'url': 'https://crossroadsamsterdam.nl/woningaanbod/verdieping-27/#6311'},
  {'type': 'woningaanbod - verdieping 27 - appartement 6312',
   'url': 'https://crossroadsamsterdam.nl/woningaanbod/verdieping-27/#6312'},
  {'type': 'woningaanbod - verdieping 3 - appartement 6318',
   'url': 'https://crossroadsamsterdam.nl/woningaanbod/verdieping-3/#6318'},
  {'type': 'woningaanbod - verdieping 3 - appartement 6319',
   'url': 'https://crossroadsamsterdam.nl/woningaanbod/verdieping-3/#6319'},
  {'type': 'woningaanbod - verdieping 3 - appartement 6321',
   'url': 'https://crossroadsamsterdam.nl/woningaanbod/verdieping-3/#6321'},
  {'type': 'wonin

In [ ]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Scrape_Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\n\n"
    
    # def get_house_information(self):

        


In [ ]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Scrape_Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [31]:
print(get_all_details("https://crossroadsamsterdam.nl/"))

Found links: {'links': [{'type': 'woningaanbod', 'url': 'https://crossroadsamsterdam.nl/woningaanbod/'}]}
Landing page:
Webpage Title:
Crossroads Amsterdam
Webpage Contents:
Toggle navigation
Home
Locatie
Woningaanbod
Voordelen
Veelgestelde vragen
Contact
Account / Inschrijven
Zoek je droom appartement
48
te huur
32
in optie
40
verhuurd
Zoek
Schrijf je nu in voor een van de appartementen!
Inschrijven
INKOMENS CHECK
Bereken snel of je in aanmerking komt voor een vrije sector huurwoning
slechts een indicatie
Verzend
Bereken
×
Inkomens check
Jouw gegevens:
Bruto maandinkomen: €
,-
Bruto maandinkomen partner: €
,-
Op basis van je opgegeven maandinkomen (en dat van jouw partner) is het bij Crossroads mogelijk om een appartement van maximaal
€
,- per maand te huren.
Dit is op basis van de ingevulde gegevens en slechts een indicatie.
Uiteraard kun je ook huren met een AOW of eigen vermogen. Kijk snel bij de
veelgestelde vragen
of daar iets over jouw situatie staat. Zo niet, neem gerust contac